![image](https://github.com/IBM/watson-machine-learning-samples/raw/master/cloud/notebooks/headers/AutoAI-Banner_Pipeline-Notebook.png)
# Pipeline 8 Notebook - AutoAI Notebook v1.16.9

Consider these tips for working with an auto-generated notebook:
- Notebook code generated using AutoAI will execute successfully. If you modify the notebook, we cannot guarantee it will run successfully.
- This pipeline is optimized for the original data set. The pipeline might fail or produce sub-optimal results if used with different data.  If you want to use a different data set, consider retraining the AutoAI experiment to generate a new pipeline. For more information, see <a href="https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/autoai-notebook.html">Cloud Platform</a>. 
- Before modifying the pipeline or trying to re-fit the pipeline, consider that the code converts dataframes to numpy arrays before fitting the pipeline (a current restriction of the preprocessor pipeline).


<a id="content"></a>
## Notebook content

This notebook contains a Scikit-learn representation of AutoAI pipeline. This notebook introduces commands for retrieving data, training the model, and testing the model. 

Some familiarity with Python is helpful. This notebook uses python 3.9 and scikit-learn 1.0.2.

## Notebook goals

-  Scikit-learn pipeline definition
-  Pipeline training 
-  Pipeline evaluation

## Contents

This notebook contains the following parts:

**[Setup](#setup)**<br>
&nbsp;&nbsp;[Package installation](#install)<br>
&nbsp;&nbsp;[AutoAI experiment metadata](#variables_definition)<br>
&nbsp;&nbsp;[Watson Machine Learning connection](#connection)<br>
**[Pipeline inspection](#inspection)** <br>
&nbsp;&nbsp;[Read training data](#read)<br>
&nbsp;&nbsp;[Train and test data split](#split)<br>
&nbsp;&nbsp;[Create pipeline](#preview_model_to_python_code)<br>
&nbsp;&nbsp;[Train pipeline model](#train)<br>
&nbsp;&nbsp;[Test pipeline model](#test_model)<br>
**[Summary and next steps](#summary_and_next_steps)**<br>
**[Copyrights](#copyrights)**

<a id="setup"></a>
# Setup

<a id="install"></a>
## Package installation
Before you use the sample code in this notebook, install the following packages:
 - ibm_watson_machine_learning,
 - autoai-libs,
 - scikit-learn,
 - snapml.


In [ ]:
!pip install -U ibm-watson-machine-learning | tail -n 1
!pip install -U autoai-libs==1.13.9 | tail -n 1
!pip install -U scikit-learn==1.0.2 | tail -n 1
!pip install -U 'snapml>=1.8.8,<1.9.0' | tail -n 1

<a id="variables_definition"></a>
## AutoAI experiment metadata
The following cell contains the training data connection details.  
**Note**: The connection might contain authorization credentials, so be careful when sharing the notebook.

In [ ]:
from ibm_watson_machine_learning.helpers import DataConnection
from ibm_watson_machine_learning.helpers import ContainerLocation

training_data_references = [
    DataConnection(
        data_asset_id='6ace2858-394d-45b3-8125-26b210d1a3e8'
    ),
]
training_result_reference = DataConnection(
    location=ContainerLocation(
        path='auto_ml/bba734b8-4554-47d8-81dd-c346590e3cdd/wml_data/7e1446fc-5f28-48d1-a203-e8ce9ca5c377/data/automl',
        model_location='auto_ml/bba734b8-4554-47d8-81dd-c346590e3cdd/wml_data/7e1446fc-5f28-48d1-a203-e8ce9ca5c377/data/automl/model.zip',
        training_status='auto_ml/bba734b8-4554-47d8-81dd-c346590e3cdd/wml_data/7e1446fc-5f28-48d1-a203-e8ce9ca5c377/training-status.json'
    )
)

The following cell contains input parameters provided to run the AutoAI experiment in Watson Studio.

In [ ]:
experiment_metadata = dict(
    prediction_type='regression',
    prediction_column='Spread',
    holdout_size=0.1,
    scoring='neg_root_mean_squared_error',
    csv_separator=',',
    random_state=33,
    max_number_of_estimators=2,
    training_data_references=training_data_references,
    training_result_reference=training_result_reference,
    deployment_url='https://us-south.ml.cloud.ibm.com',
    project_id='011c9d81-93cd-4342-9b0c-a2e4af1e5ec2',
    drop_duplicates=True
)

## Set `n_jobs` parameter to the number of available CPUs

In [ ]:
import os, ast
CPU_NUMBER = 2
if 'RUNTIME_HARDWARE_SPEC' in os.environ:
    CPU_NUMBER = int(ast.literal_eval(os.environ['RUNTIME_HARDWARE_SPEC'])['num_cpu'])

<a id="connection"></a>
## Watson Machine Learning connection

This cell defines the credentials required to work with the Watson Machine Learning service.

**Action:** Please provide IBM Cloud apikey following [docs](https://cloud.ibm.com/docs/account?topic=account-userapikey).

In [ ]:
api_key = 'PUT_YOUR_APIKEY_HERE'

In [ ]:
wml_credentials = {
    "apikey": api_key,
    "url": experiment_metadata['deployment_url']
}

In [ ]:
from ibm_watson_machine_learning import APIClient

wml_client = APIClient(wml_credentials)

if 'space_id' in experiment_metadata:
    wml_client.set.default_space(experiment_metadata['space_id'])
else:
    wml_client.set.default_project(experiment_metadata['project_id'])
    
training_data_references[0].set_client(wml_client)


<a id="inspection"></a>
# Pipeline inspection

<a id="read"></a>
## Read training data

Retrieve training dataset from AutoAI experiment as pandas DataFrame.

**Note**: If reading data results in an error, provide data as Pandas DataFrame object, for example, reading .CSV file with `pandas.read_csv()`. It may be necessary to use methods for initial data pre-processing like: e.g. `DataFrame.dropna()`, `DataFrame.drop_duplicates()`, `DataFrame.sample()`.

In [ ]:
train_X, test_X, train_y, test_y = training_data_references[0].read(experiment_metadata=experiment_metadata, with_holdout_split=True, use_flight=False)

<a id="preview_model_to_python_code"></a>
## Create pipeline
In the next cell, you can find the Scikit-learn definition of the selected AutoAI pipeline.

#### Import statements.

In [ ]:
from autoai_libs.transformers.date_time import DateTransformer
from autoai_libs.transformers.exportable import NumpyColumnSelector
from autoai_libs.transformers.exportable import CompressStrings
from autoai_libs.transformers.exportable import NumpyReplaceMissingValues
from autoai_libs.transformers.exportable import NumpyReplaceUnknownValues
from autoai_libs.transformers.exportable import boolean2float
from autoai_libs.transformers.exportable import CatImputer
from autoai_libs.transformers.exportable import CatEncoder
import numpy as np
from autoai_libs.transformers.exportable import float32_transform
from sklearn.pipeline import make_pipeline
from autoai_libs.transformers.exportable import FloatStr2Float
from autoai_libs.transformers.exportable import NumImputer
from autoai_libs.transformers.exportable import OptStandardScaler
from sklearn.pipeline import make_union
from autoai_libs.transformers.exportable import NumpyPermuteArray
from autoai_libs.cognito.transforms.transform_utils import TA2
import autoai_libs.utils.fc_methods
from autoai_libs.cognito.transforms.transform_utils import FS1
from snapml import SnapRandomForestRegressor

#### Pre-processing & Estimator.

In [ ]:
date_transformer_0 = DateTransformer(
    column_headers_list=[
        "Week", "Day", "Date", "Time", "full_team_name", "full_team_name_def",
        "home", "prf_opp_PA", "prf_opp_Yds", "prf_opp_TO_Ply",
        "prf_opp_TO_Y/P", "prf_opp_TO_TO", "prf_opp_FL", "prf_opp_1stD",
        "prf_opp_passingCmp", "prf_opp_passingAtt", "prf_opp_passingYds",
        "prf_opp_passingTD", "prf_opp_passingInt", "prf_opp_passingNY/A",
        "prf_opp_passing1stD", "prf_opp_rushingAtt", "prf_opp_rushingYds",
        "prf_opp_rushingTD", "prf_opp_rushingY/A", "prf_opp_rushing1stD",
        "prf_opp_rushingPen", "prf_opp_rushingYds.1", "prf_opp_rushing1stPy",
        "prf_opp_Sc", "prf_opp_TO", "prf_opp_EXP", "Win", "Loss", "Tie",
        "SOS", "PR", "dvoa_off_Adjusted Line Yards",
        "dvoa_off_Running Back Yards", "dvoa_off_Power Success Rate",
        "dvoa_off_Open Field Yards", "dvoa_off_Second Level Yards",
        "dvoa_off_Stuffed Rate", "dvoa_off_Adjusted Sack Rate",
        "dvoa_off_Sacks", "dvoa_def_Adjusted Line Yards",
        "dvoa_def_Running Back Yards", "dvoa_def_Power Success Rate",
        "dvoa_def_Open Field Yards", "dvoa_def_Second Level Yards",
        "dvoa_def_Stuffed Rate", "dvoa_def_Adjusted Sack Rate",
        "dvoa_def_Sacks", "hfa", "def_prf_opp_PA", "def_prf_opp_Yds",
        "def_prf_opp_TO_Ply", "def_prf_opp_TO_Y/P", "def_prf_opp_TO_TO",
        "def_prf_opp_FL", "def_prf_opp_1stD", "def_prf_opp_passingCmp",
        "def_prf_opp_passingAtt", "def_prf_opp_passingYds",
        "def_prf_opp_passingTD", "def_prf_opp_passingInt",
        "def_prf_opp_passingNY/A", "def_prf_opp_passing1stD",
        "def_prf_opp_rushingAtt", "def_prf_opp_rushingYds",
        "def_prf_opp_rushingTD", "def_prf_opp_rushingY/A",
        "def_prf_opp_rushing1stD", "def_prf_opp_rushingPen",
        "def_prf_opp_rushingYds.1", "def_prf_opp_rushing1stPy",
        "def_prf_opp_Sc", "def_prf_opp_TO", "def_prf_opp_EXP", "def_Win",
        "def_Loss", "def_Tie", "def_SOS", "def_PR",
        "def_dvoa_off_Adjusted Line Yards", "def_dvoa_off_Running Back Yards",
        "def_dvoa_off_Power Success Rate", "def_dvoa_off_Open Field Yards",
        "def_dvoa_off_Second Level Yards", "def_dvoa_off_Stuffed Rate",
        "def_dvoa_off_Adjusted Sack Rate", "def_dvoa_off_Sacks",
        "def_dvoa_def_Adjusted Line Yards", "def_dvoa_def_Running Back Yards",
        "def_dvoa_def_Power Success Rate", "def_dvoa_def_Open Field Yards",
        "def_dvoa_def_Second Level Yards", "def_dvoa_def_Stuffed Rate",
        "def_dvoa_def_Adjusted Sack Rate", "def_dvoa_def_Sacks", "def_hfa",
    ],
    delete_source_columns=False,
    missing_values_reference_list=["?", "", "-", float("nan")],
    options=["all"],
)
numpy_column_selector_0 = NumpyColumnSelector(
    columns=[
        0, 1, 2, 3, 4, 5, 6, 11, 12, 17, 18, 20, 23, 25, 26, 28, 32, 33, 34,
        39, 44, 47, 52, 58, 59, 64, 65, 67, 70, 72, 73, 75, 79, 80, 81, 91,
        99, 101, 102, 103, 104, 105, 106, 107, 108, 109,
    ]
)
compress_strings = CompressStrings(
    compress_type="hash",
    dtypes_list=[
        "int_num", "char_str", "char_str", "char_str", "char_str", "char_str",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "float_int_num", "float_int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num",
    ],
    missing_values_reference_list=["", "-", "?", float("nan")],
    misslist_list=[
        [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [],
        [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [],
        [], [], [], [], [], [], [], [], [], [], [], [],
    ],
)
numpy_replace_missing_values_0 = NumpyReplaceMissingValues(
    missing_values=[], filling_values=float("nan")
)
numpy_replace_unknown_values = NumpyReplaceUnknownValues(
    filling_values=float("nan"),
    filling_values_list=[
        float("nan"), float("nan"), float("nan"), float("nan"), float("nan"),
        float("nan"), float("nan"), float("nan"), float("nan"), float("nan"),
        float("nan"), float("nan"), float("nan"), float("nan"), float("nan"),
        float("nan"), float("nan"), float("nan"), float("nan"), float("nan"),
        float("nan"), float("nan"), float("nan"), float("nan"), float("nan"),
        float("nan"), float("nan"), float("nan"), float("nan"), float("nan"),
        float("nan"), float("nan"), float("nan"), float("nan"), float("nan"),
        float("nan"), float("nan"), 100001, 100001, float("nan"),
        float("nan"), float("nan"), float("nan"), float("nan"), float("nan"),
        float("nan"),
    ],
    missing_values_reference_list=["", "-", "?", float("nan")],
)
cat_imputer = CatImputer(
    missing_values=float("nan"),
    sklearn_version_family="1",
    strategy="most_frequent",
)
cat_encoder = CatEncoder(
    encoding="ordinal",
    categories="auto",
    dtype=np.float64,
    handle_unknown="error",
    sklearn_version_family="1",
)
pipeline_0 = make_pipeline(
    date_transformer_0,
    numpy_column_selector_0,
    compress_strings,
    numpy_replace_missing_values_0,
    numpy_replace_unknown_values,
    boolean2float(),
    cat_imputer,
    cat_encoder,
    float32_transform(),
)
date_transformer_1 = DateTransformer(
    column_headers_list=[
        "Week", "Day", "Date", "Time", "full_team_name", "full_team_name_def",
        "home", "prf_opp_PA", "prf_opp_Yds", "prf_opp_TO_Ply",
        "prf_opp_TO_Y/P", "prf_opp_TO_TO", "prf_opp_FL", "prf_opp_1stD",
        "prf_opp_passingCmp", "prf_opp_passingAtt", "prf_opp_passingYds",
        "prf_opp_passingTD", "prf_opp_passingInt", "prf_opp_passingNY/A",
        "prf_opp_passing1stD", "prf_opp_rushingAtt", "prf_opp_rushingYds",
        "prf_opp_rushingTD", "prf_opp_rushingY/A", "prf_opp_rushing1stD",
        "prf_opp_rushingPen", "prf_opp_rushingYds.1", "prf_opp_rushing1stPy",
        "prf_opp_Sc", "prf_opp_TO", "prf_opp_EXP", "Win", "Loss", "Tie",
        "SOS", "PR", "dvoa_off_Adjusted Line Yards",
        "dvoa_off_Running Back Yards", "dvoa_off_Power Success Rate",
        "dvoa_off_Open Field Yards", "dvoa_off_Second Level Yards",
        "dvoa_off_Stuffed Rate", "dvoa_off_Adjusted Sack Rate",
        "dvoa_off_Sacks", "dvoa_def_Adjusted Line Yards",
        "dvoa_def_Running Back Yards", "dvoa_def_Power Success Rate",
        "dvoa_def_Open Field Yards", "dvoa_def_Second Level Yards",
        "dvoa_def_Stuffed Rate", "dvoa_def_Adjusted Sack Rate",
        "dvoa_def_Sacks", "hfa", "def_prf_opp_PA", "def_prf_opp_Yds",
        "def_prf_opp_TO_Ply", "def_prf_opp_TO_Y/P", "def_prf_opp_TO_TO",
        "def_prf_opp_FL", "def_prf_opp_1stD", "def_prf_opp_passingCmp",
        "def_prf_opp_passingAtt", "def_prf_opp_passingYds",
        "def_prf_opp_passingTD", "def_prf_opp_passingInt",
        "def_prf_opp_passingNY/A", "def_prf_opp_passing1stD",
        "def_prf_opp_rushingAtt", "def_prf_opp_rushingYds",
        "def_prf_opp_rushingTD", "def_prf_opp_rushingY/A",
        "def_prf_opp_rushing1stD", "def_prf_opp_rushingPen",
        "def_prf_opp_rushingYds.1", "def_prf_opp_rushing1stPy",
        "def_prf_opp_Sc", "def_prf_opp_TO", "def_prf_opp_EXP", "def_Win",
        "def_Loss", "def_Tie", "def_SOS", "def_PR",
        "def_dvoa_off_Adjusted Line Yards", "def_dvoa_off_Running Back Yards",
        "def_dvoa_off_Power Success Rate", "def_dvoa_off_Open Field Yards",
        "def_dvoa_off_Second Level Yards", "def_dvoa_off_Stuffed Rate",
        "def_dvoa_off_Adjusted Sack Rate", "def_dvoa_off_Sacks",
        "def_dvoa_def_Adjusted Line Yards", "def_dvoa_def_Running Back Yards",
        "def_dvoa_def_Power Success Rate", "def_dvoa_def_Open Field Yards",
        "def_dvoa_def_Second Level Yards", "def_dvoa_def_Stuffed Rate",
        "def_dvoa_def_Adjusted Sack Rate", "def_dvoa_def_Sacks", "def_hfa",
    ],
    delete_source_columns=False,
    missing_values_reference_list=["?", "", "-", float("nan")],
    options=["all"],
)
numpy_column_selector_1 = NumpyColumnSelector(
    columns=[
        7, 8, 9, 10, 13, 14, 15, 16, 19, 21, 22, 24, 27, 29, 30, 31, 35, 36,
        37, 38, 40, 41, 42, 43, 45, 46, 48, 49, 50, 51, 53, 54, 55, 56, 57,
        60, 61, 62, 63, 66, 68, 69, 71, 74, 76, 77, 78, 82, 83, 84, 85, 86,
        87, 88, 89, 90, 92, 93, 94, 95, 96, 97, 98, 100,
    ]
)
float_str2_float = FloatStr2Float(
    dtypes_list=[
        "int_num", "int_num", "int_num", "float_num", "int_num", "int_num",
        "int_num", "int_num", "float_num", "int_num", "int_num", "float_num",
        "int_num", "float_num", "float_num", "float_num", "float_num",
        "float_num", "float_num", "float_num", "float_num", "float_num",
        "float_num", "float_num", "float_num", "float_num", "float_num",
        "float_num", "float_num", "float_num", "float_num", "int_num",
        "int_num", "int_num", "float_num", "int_num", "int_num", "int_num",
        "int_num", "float_num", "int_num", "int_num", "float_num", "int_num",
        "float_num", "float_num", "float_num", "float_num", "float_num",
        "float_num", "float_num", "float_num", "float_num", "float_num",
        "float_num", "float_num", "float_num", "float_num", "float_num",
        "float_num", "float_num", "float_num", "float_num", "float_num",
    ],
    missing_values_reference_list=[],
)
numpy_replace_missing_values_1 = NumpyReplaceMissingValues(
    missing_values=[], filling_values=float("nan")
)
num_imputer = NumImputer(missing_values=float("nan"), strategy="median")
opt_standard_scaler = OptStandardScaler(use_scaler_flag=False)
pipeline_1 = make_pipeline(
    date_transformer_1,
    numpy_column_selector_1,
    float_str2_float,
    numpy_replace_missing_values_1,
    num_imputer,
    opt_standard_scaler,
    float32_transform(),
)
union = make_union(pipeline_0, pipeline_1)
numpy_permute_array = NumpyPermuteArray(
    axis=0,
    permutation_indices=[
        0, 1, 2, 3, 4, 5, 6, 11, 12, 17, 18, 20, 23, 25, 26, 28, 32, 33, 34,
        39, 44, 47, 52, 58, 59, 64, 65, 67, 70, 72, 73, 75, 79, 80, 81, 91,
        99, 101, 102, 103, 104, 105, 106, 107, 108, 109, 7, 8, 9, 10, 13, 14,
        15, 16, 19, 21, 22, 24, 27, 29, 30, 31, 35, 36, 37, 38, 40, 41, 42,
        43, 45, 46, 48, 49, 50, 51, 53, 54, 55, 56, 57, 60, 61, 62, 63, 66,
        68, 69, 71, 74, 76, 77, 78, 82, 83, 84, 85, 86, 87, 88, 89, 90, 92,
        93, 94, 95, 96, 97, 98, 100,
    ],
)
ta2_0 = TA2(
    fun=np.add,
    name="sum",
    datatypes1=[
        "intc", "intp", "int_", "uint8", "uint16", "uint32", "uint64", "int8",
        "int16", "int32", "int64", "short", "long", "longlong", "float16",
        "float32", "float64",
    ],
    feat_constraints1=[autoai_libs.utils.fc_methods.is_not_categorical],
    datatypes2=[
        "intc", "intp", "int_", "uint8", "uint16", "uint32", "uint64", "int8",
        "int16", "int32", "int64", "short", "long", "longlong", "float16",
        "float32", "float64",
    ],
    feat_constraints2=[autoai_libs.utils.fc_methods.is_not_categorical],
    col_names=[
        "Week", "Day", "Date", "Time", "full_team_name", "full_team_name_def",
        "home", "prf_opp_PA", "prf_opp_Yds", "prf_opp_TO_Ply",
        "prf_opp_TO_Y/P", "prf_opp_TO_TO", "prf_opp_FL", "prf_opp_1stD",
        "prf_opp_passingCmp", "prf_opp_passingAtt", "prf_opp_passingYds",
        "prf_opp_passingTD", "prf_opp_passingInt", "prf_opp_passingNY/A",
        "prf_opp_passing1stD", "prf_opp_rushingAtt", "prf_opp_rushingYds",
        "prf_opp_rushingTD", "prf_opp_rushingY/A", "prf_opp_rushing1stD",
        "prf_opp_rushingPen", "prf_opp_rushingYds.1", "prf_opp_rushing1stPy",
        "prf_opp_Sc", "prf_opp_TO", "prf_opp_EXP", "Win", "Loss", "Tie",
        "SOS", "PR", "dvoa_off_Adjusted Line Yards",
        "dvoa_off_Running Back Yards", "dvoa_off_Power Success Rate",
        "dvoa_off_Open Field Yards", "dvoa_off_Second Level Yards",
        "dvoa_off_Stuffed Rate", "dvoa_off_Adjusted Sack Rate",
        "dvoa_off_Sacks", "dvoa_def_Adjusted Line Yards",
        "dvoa_def_Running Back Yards", "dvoa_def_Power Success Rate",
        "dvoa_def_Open Field Yards", "dvoa_def_Second Level Yards",
        "dvoa_def_Stuffed Rate", "dvoa_def_Adjusted Sack Rate",
        "dvoa_def_Sacks", "hfa", "def_prf_opp_PA", "def_prf_opp_Yds",
        "def_prf_opp_TO_Ply", "def_prf_opp_TO_Y/P", "def_prf_opp_TO_TO",
        "def_prf_opp_FL", "def_prf_opp_1stD", "def_prf_opp_passingCmp",
        "def_prf_opp_passingAtt", "def_prf_opp_passingYds",
        "def_prf_opp_passingTD", "def_prf_opp_passingInt",
        "def_prf_opp_passingNY/A", "def_prf_opp_passing1stD",
        "def_prf_opp_rushingAtt", "def_prf_opp_rushingYds",
        "def_prf_opp_rushingTD", "def_prf_opp_rushingY/A",
        "def_prf_opp_rushing1stD", "def_prf_opp_rushingPen",
        "def_prf_opp_rushingYds.1", "def_prf_opp_rushing1stPy",
        "def_prf_opp_Sc", "def_prf_opp_TO", "def_prf_opp_EXP", "def_Win",
        "def_Loss", "def_Tie", "def_SOS", "def_PR",
        "def_dvoa_off_Adjusted Line Yards", "def_dvoa_off_Running Back Yards",
        "def_dvoa_off_Power Success Rate", "def_dvoa_off_Open Field Yards",
        "def_dvoa_off_Second Level Yards", "def_dvoa_off_Stuffed Rate",
        "def_dvoa_off_Adjusted Sack Rate", "def_dvoa_off_Sacks",
        "def_dvoa_def_Adjusted Line Yards", "def_dvoa_def_Running Back Yards",
        "def_dvoa_def_Power Success Rate", "def_dvoa_def_Open Field Yards",
        "def_dvoa_def_Second Level Yards", "def_dvoa_def_Stuffed Rate",
        "def_dvoa_def_Adjusted Sack Rate", "def_dvoa_def_Sacks", "def_hfa",
        "NewDateFeature_0_FloatTimestamp(Date)",
        "NewDateFeature_1_FloatTimestamp(Time)",
        "NewDateFeature_0_Month(Date)", "NewDateFeature_0_Week(Date)",
        "NewDateFeature_0_DayOfYear(Date)",
        "NewDateFeature_0_DayOfMonth(Date)",
        "NewDateFeature_0_DayOfWeek(Date)", "NewDateFeature_1_Hour(Time)",
        "NewDateFeature_1_Minute(Time)",
    ],
    col_dtypes=[
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"),
    ],
)
fs1_0 = FS1(
    cols_ids_must_keep=range(0, 110),
    additional_col_count_to_keep=36,
    ptype="regression",
)
ta2_1 = TA2(
    fun=np.multiply,
    name="product",
    datatypes1=[
        "intc", "intp", "int_", "uint8", "uint16", "uint32", "uint64", "int8",
        "int16", "int32", "int64", "short", "long", "longlong", "float16",
        "float32", "float64",
    ],
    feat_constraints1=[autoai_libs.utils.fc_methods.is_not_categorical],
    datatypes2=[
        "intc", "intp", "int_", "uint8", "uint16", "uint32", "uint64", "int8",
        "int16", "int32", "int64", "short", "long", "longlong", "float16",
        "float32", "float64",
    ],
    feat_constraints2=[autoai_libs.utils.fc_methods.is_not_categorical],
    col_names=[
        "Week", "Day", "Date", "Time", "full_team_name", "full_team_name_def",
        "home", "prf_opp_PA", "prf_opp_Yds", "prf_opp_TO_Ply",
        "prf_opp_TO_Y/P", "prf_opp_TO_TO", "prf_opp_FL", "prf_opp_1stD",
        "prf_opp_passingCmp", "prf_opp_passingAtt", "prf_opp_passingYds",
        "prf_opp_passingTD", "prf_opp_passingInt", "prf_opp_passingNY/A",
        "prf_opp_passing1stD", "prf_opp_rushingAtt", "prf_opp_rushingYds",
        "prf_opp_rushingTD", "prf_opp_rushingY/A", "prf_opp_rushing1stD",
        "prf_opp_rushingPen", "prf_opp_rushingYds.1", "prf_opp_rushing1stPy",
        "prf_opp_Sc", "prf_opp_TO", "prf_opp_EXP", "Win", "Loss", "Tie",
        "SOS", "PR", "dvoa_off_Adjusted Line Yards",
        "dvoa_off_Running Back Yards", "dvoa_off_Power Success Rate",
        "dvoa_off_Open Field Yards", "dvoa_off_Second Level Yards",
        "dvoa_off_Stuffed Rate", "dvoa_off_Adjusted Sack Rate",
        "dvoa_off_Sacks", "dvoa_def_Adjusted Line Yards",
        "dvoa_def_Running Back Yards", "dvoa_def_Power Success Rate",
        "dvoa_def_Open Field Yards", "dvoa_def_Second Level Yards",
        "dvoa_def_Stuffed Rate", "dvoa_def_Adjusted Sack Rate",
        "dvoa_def_Sacks", "hfa", "def_prf_opp_PA", "def_prf_opp_Yds",
        "def_prf_opp_TO_Ply", "def_prf_opp_TO_Y/P", "def_prf_opp_TO_TO",
        "def_prf_opp_FL", "def_prf_opp_1stD", "def_prf_opp_passingCmp",
        "def_prf_opp_passingAtt", "def_prf_opp_passingYds",
        "def_prf_opp_passingTD", "def_prf_opp_passingInt",
        "def_prf_opp_passingNY/A", "def_prf_opp_passing1stD",
        "def_prf_opp_rushingAtt", "def_prf_opp_rushingYds",
        "def_prf_opp_rushingTD", "def_prf_opp_rushingY/A",
        "def_prf_opp_rushing1stD", "def_prf_opp_rushingPen",
        "def_prf_opp_rushingYds.1", "def_prf_opp_rushing1stPy",
        "def_prf_opp_Sc", "def_prf_opp_TO", "def_prf_opp_EXP", "def_Win",
        "def_Loss", "def_Tie", "def_SOS", "def_PR",
        "def_dvoa_off_Adjusted Line Yards", "def_dvoa_off_Running Back Yards",
        "def_dvoa_off_Power Success Rate", "def_dvoa_off_Open Field Yards",
        "def_dvoa_off_Second Level Yards", "def_dvoa_off_Stuffed Rate",
        "def_dvoa_off_Adjusted Sack Rate", "def_dvoa_off_Sacks",
        "def_dvoa_def_Adjusted Line Yards", "def_dvoa_def_Running Back Yards",
        "def_dvoa_def_Power Success Rate", "def_dvoa_def_Open Field Yards",
        "def_dvoa_def_Second Level Yards", "def_dvoa_def_Stuffed Rate",
        "def_dvoa_def_Adjusted Sack Rate", "def_dvoa_def_Sacks", "def_hfa",
        "NewDateFeature_0_FloatTimestamp(Date)",
        "NewDateFeature_1_FloatTimestamp(Time)",
        "NewDateFeature_0_Month(Date)", "NewDateFeature_0_Week(Date)",
        "NewDateFeature_0_DayOfYear(Date)",
        "NewDateFeature_0_DayOfMonth(Date)",
        "NewDateFeature_0_DayOfWeek(Date)", "NewDateFeature_1_Hour(Time)",
        "NewDateFeature_1_Minute(Time)", "sum(Date__def_prf_opp_PA)",
        "sum(prf_opp_PA__prf_opp_rushingPen)",
        "sum(prf_opp_PA__dvoa_def_Running Back Yards)",
        "sum(prf_opp_PA__dvoa_def_Power Success Rate)",
        "sum(prf_opp_PA__dvoa_def_Open Field Yards)",
        "sum(prf_opp_PA__dvoa_def_Stuffed Rate)",
        "sum(prf_opp_PA__dvoa_def_Adjusted Sack Rate)",
        "sum(prf_opp_PA__NewDateFeature_0_DayOfYear(Date))",
        "sum(prf_opp_rushingPen__def_prf_opp_PA)",
        "sum(PR__dvoa_def_Adjusted Line Yards)",
        "sum(dvoa_off_Adjusted Line Yards__def_prf_opp_PA)",
        "sum(dvoa_off_Open Field Yards__prf_opp_PA)",
        "sum(dvoa_off_Open Field Yards__PR)",
        "sum(dvoa_off_Open Field Yards__def_PR)",
        "sum(dvoa_def_Adjusted Line Yards__def_prf_opp_PA)",
        "sum(dvoa_def_Adjusted Line Yards__def_PR)",
        "sum(dvoa_def_Open Field Yards__def_PR)",
        "sum(dvoa_def_Adjusted Sack Rate__def_prf_opp_Sc)",
        "sum(def_prf_opp_PA__Date)",
        "sum(def_prf_opp_PA__dvoa_off_Stuffed Rate)",
        "sum(def_prf_opp_PA__dvoa_off_Adjusted Sack Rate)",
        "sum(def_prf_opp_PA__dvoa_def_Adjusted Sack Rate)",
        "sum(hfa__def_prf_opp_TO)",
        "sum(def_prf_opp_PA__def_dvoa_def_Adjusted Line Yards)",
        "sum(def_prf_opp_PA__def_dvoa_def_Open Field Yards)",
        "sum(def_prf_opp_PA__def_hfa)",
        "sum(def_prf_opp_Sc__dvoa_def_Adjusted Sack Rate)",
        "sum(def_prf_opp_EXP__prf_opp_rushingAtt)", "sum(def_PR__prf_opp_PA)",
        "sum(def_PR__dvoa_off_Open Field Yards)",
        "sum(def_PR__def_dvoa_off_Adjusted Line Yards)",
        "sum(def_PR__def_dvoa_off_Running Back Yards)",
        "sum(def_PR__def_dvoa_def_Power Success Rate)",
        "sum(def_dvoa_off_Open Field Yards__def_prf_opp_PA)",
        "sum(def_dvoa_def_Power Success Rate__PR)",
        "sum(def_dvoa_def_Second Level Yards__def_PR)",
    ],
    col_dtypes=[
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"),
    ],
)
fs1_1 = FS1(
    cols_ids_must_keep=range(0, 110),
    additional_col_count_to_keep=36,
    ptype="regression",
)
snap_random_forest_regressor = SnapRandomForestRegressor(
    gpu_ids=np.array([0], dtype=np.uint32),
    max_depth=3,
    max_features=0.10004256176720457,
    n_estimators=37,
    n_jobs=CPU_NUMBER,
    random_state=33,
)


#### Pipeline.

In [ ]:
pipeline = make_pipeline(
    union,
    numpy_permute_array,
    ta2_0,
    fs1_0,
    ta2_1,
    fs1_1,
    snap_random_forest_regressor,
)

<a id="train"></a>
## Train pipeline model


### Define scorer from the optimization metric
This cell constructs the cell scorer based on the experiment metadata.

In [ ]:
from sklearn.metrics import get_scorer

scorer = get_scorer(experiment_metadata['scoring'])

<a id="test_model"></a>
### Fit pipeline model
In this cell, the pipeline is fitted.

In [ ]:
pipeline.fit(train_X.values, train_y.values.ravel());

<a id="test_model"></a>
## Test pipeline model

Score the fitted pipeline with the generated scorer using the holdout dataset.

In [ ]:
score = scorer(pipeline, test_X.values, test_y.values)
print(score)

In [ ]:
pipeline.predict(test_X.values)

<a id="summary_and_next_steps"></a>
# Summary and next steps
You successfully completed this notebook!
You learned how to use AutoAI pipeline definition to train the model.
Check out our [Online Documentation](https://www.ibm.com/cloud/watson-studio/autoai) for more samples, tutorials, documentation, how-tos, and blog posts.

<a id="copyrights"></a>
### Copyrights

Licensed Materials - Copyright © 2022 IBM. This notebook and its source code are released under the terms of the ILAN License. Use, duplication disclosure restricted by GSA ADP Schedule Contract with IBM Corp.

**Note:** The auto-generated notebooks are subject to the International License Agreement for Non-Warranted Programs (or equivalent) and License Information document for Watson Studio Auto-generated Notebook (License Terms), such agreements located in the link below. Specifically, the Source Components and Sample Materials clause included in the License Information document for Watson Studio Auto-generated Notebook applies to the auto-generated notebooks.  

By downloading, copying, accessing, or otherwise using the materials, you agree to the <a href="http://www14.software.ibm.com/cgi-bin/weblap/lap.pl?li_formnum=L-AMCU-BYC7LF">License Terms</a>

___